In [1]:
%pip install optuna==3.6.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.0 MB/s eta 0:00:00


In [2]:
pip install --upgrade lightgbm

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer

import lightgbm as lgb


import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

import pickle

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [4]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
dataset_path = '/content/drive/MyDrive/MaestDataMining/Especializacion/2C/DMEF/Datasets/'
dataset_file = 'competencia_02_Ternaria_PaOpt.csv'

data = pd.read_csv(dataset_path + dataset_file)

<ipython-input-5-944a61a0f8c1>:4: DtypeWarning: Columns (155) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(dataset_path + dataset_file)


In [6]:
# Eliminar la columna 'X'
data = data.drop(columns=['X'], errors='ignore')


In [7]:
data['foto_mes'] = data['foto_mes'].astype(str)  # Asegura que 'foto_mes' sea tipo string

# Filtrar y contar para cada mes por separado
meses_filtrados = ["202108", "202107", "202106", "202105"]

for mes in meses_filtrados:
    # Filtrar los datos para el mes específico
    data_mes = data[data['foto_mes'] == mes]

    # Comprobar si hay datos para el mes
    if not data_mes.empty:
        # Contar la cantidad de casos por categoría en 'clase_ternaria'
        conteo_clases = data_mes['clase_ternaria'].value_counts()

        # Mostrar el resultado para el mes
        print(f"Conteo de clases para {mes}:")
        print(conteo_clases)
    else:
        print(f"No hay datos para el mes {mes}")
    print("\n")


    # Junio es el último mes con datos para entrenar
    # Queremos predecir agosto

Conteo de clases para 202108:
Series([], Name: count, dtype: int64)


Conteo de clases para 202107:
clase_ternaria
BAJA+1    1075
Name: count, dtype: int64


Conteo de clases para 202106:
clase_ternaria
CONTINUA    162889
BAJA+2        1074
BAJA+1         913
Name: count, dtype: int64


Conteo de clases para 202105:
clase_ternaria
CONTINUA    162520
BAJA+1        1192
BAJA+2         911
Name: count, dtype: int64




In [8]:
#Se está rompiendo más adelante, voy a borrar más meses. Dejo desde agsto 2020 (ahora dejo julio pero solo para calcular el lag y dlag de agosto 2020)

# Filtrar los datos para mantener solo los meses a partir de agosto 2020
data = data[(data['foto_mes'] >= "202007")]

In [9]:
#Quito columnas con 1er cuartil y mediana 0, porque se rompe la ram si meto todo
omit_columns = [
    "cliente_vip", "internet", "mcuenta_corriente_adicional", "mcaja_ahorro_adicional",
    "ctarjeta_master_transacciones", "mtarjeta_master_consumo", "cprestamos_personales", "mprestamos_personales",
    "cprestamos_prendarios", "mprestamos_prendarios", "cprestamos_hipotecarios", "mprestamos_hipotecarios",
    "cplazo_fijo", "mplazo_fijo_dolares", "mplazo_fijo_pesos", "cinversion1", "minversion1_pesos", "minversion1_dolares",
    "cinversion2", "minversion2", "cseguro_vida", "cseguro_auto", "cseguro_vivienda", "cseguro_accidentes_personales",
    "ccaja_seguridad", "mpayroll2", "cpayroll2_trx", "ccuenta_debitos_automaticos", "mcuenta_debitos_automaticos",
    "ctarjeta_master_debitos_automaticos", "mttarjeta_master_debitos_automaticos", "cpagodeservicios", "mpagodeservicios",
    "cpagomiscuentas", "mpagomiscuentas", "ccajeros_propios_descuentos", "mcajeros_propios_descuentos",
    "ctarjeta_visa_descuentos", "mtarjeta_visa_descuentos", "ctarjeta_master_descuentos", "mtarjeta_master_descuentos",
    "ccomisiones_mantenimiento", "mcomisiones_mantenimiento", "cforex", "cforex_buy", "mforex_buy", "cforex_sell",
    "mforex_sell", "cextraccion_autoservicio", "mextraccion_autoservicio", "ccheques_depositados",
    "mcheques_depositados", "ccheques_emitidos", "mcheques_emitidos", "ccheques_depositados_rechazados",
    "mcheques_depositados_rechazados", "ccheques_emitidos_rechazados", "mcheques_emitidos_rechazados", "tcallcenter",
    "ccallcenter_transacciones", "ccajas_transacciones", "ccajas_consultas", "ccajas_depositos", "ccajas_extracciones",
    "ccajas_otras", "catm_trx", "matm", "catm_trx_other", "matm_other", "tmobile_app", "Master_delinquency",
    "Master_status", "Master_msaldototal", "Master_msaldopesos", "Master_msaldodolares", "Master_mconsumosdolares",
    "Master_madelantopesos", "Master_madelantodolares", "Master_mpagado", "Master_mpagosdolares",
    "Master_cadelantosefectivo", "Master_mpagominimo", "Visa_delinquency", "Visa_status", "Visa_Finiciomora",
    "Visa_msaldodolares", "Visa_mconsumosdolares", "Visa_madelantopesos", "Visa_madelantodolares", "Visa_mpagado",
    "Visa_mpagosdolares", "Visa_cadelantosefectivo"
]

# Eliminar columnas
data = data.drop(columns=omit_columns)

In [10]:
# Obtener y mostrar los nombres de las columnas del DataFrame
column_names = data.columns.tolist()

# Imprimir los nombres de las columnas
print("Nombres de las columnas del DataFrame:")
print(column_names)

Nombres de las columnas del DataFrame:
['numero_de_cliente', 'foto_mes', 'active_quarter', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad', 'mrentabilidad_annual', 'mcomisiones', 'mactivos_margen', 'mpasivos_margen', 'cproductos', 'tcuentas', 'ccuenta_corriente', 'mcuenta_corriente', 'ccaja_ahorro', 'mcaja_ahorro', 'mcaja_ahorro_dolares', 'cdescubierto_preacordado', 'mcuentas_saldo', 'ctarjeta_debito', 'ctarjeta_debito_transacciones', 'mautoservicio', 'ctarjeta_visa', 'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo', 'ctarjeta_master', 'cpayroll_trx', 'mpayroll', 'ctarjeta_visa_debitos_automaticos', 'mttarjeta_visa_debitos_automaticos', 'ccomisiones_otras', 'mcomisiones_otras', 'ctransferencias_recibidas', 'mtransferencias_recibidas', 'ctransferencias_emitidas', 'mtransferencias_emitidas', 'thomebanking', 'chomebanking_transacciones', 'ctrx_quarter', 'cmobile_app_trx', 'Master_mfinanciacion_limite', 'Master_Fvencimiento', 'Master_Finiciomora', 'Master_mconsumospesos', 'Mast

In [11]:
# Sigo sacando columnas, porque si no no puedo correr
columns_to_keep = [
    'clase_ternaria', 'numero_de_cliente', 'foto_mes',
    'mpayroll', 'mcuenta_corriente',
    'mrentabilidad', 'mrentabilidad_annual', 'mcomisiones',
    'ctrx_quarter', 'mtransferencias_emitidas',
    'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo', 'cproductos'
]

# Seleccionar únicamente las columnas deseadas. Uso estas, más adelante uso más cuando tenga mejor ram
data = data[columns_to_keep]

In [12]:
# Calcular los percentiles, omitiendo las columnas esas
for col in data.columns:
    if col not in ['numero_de_cliente', 'foto_mes', 'clase_ternaria']:
        data[f'percentile_{col}'] = data[col].rank(pct=True)

In [13]:
print(data.shape)
data.head()

(2265732, 23)


,clase_ternaria,numero_de_cliente,foto_mes,mpayroll,mcuenta_corriente,mrentabilidad,mrentabilidad_annual,mcomisiones,ctrx_quarter,mtransferencias_emitidas,...,percentile_mpayroll,percentile_mcuenta_corriente,percentile_mrentabilidad,percentile_mrentabilidad_annual,percentile_mcomisiones,percentile_ctrx_quarter,percentile_mtransferencias_emitidas,percentile_ctarjeta_visa_transacciones,percentile_mtarjeta_visa_consumo,percentile_cproductos
0,CONTINUA,249221109,202007,64515.00,0.00,1550.36,15232.06,1623.85,161,70164.19,...,0.613643,0.665010,0.559992,0.528019,0.702433,0.779994,0.847988,0.660618,0.500427,0.656463
1,CONTINUA,249221468,202007,0.00,-2423.81,893.83,20716.22,684.18,135,10557.00,...,0.235768,0.157599,0.438132,0.622306,0.449120,0.677833,0.552508,0.821192,0.773789,0.656463
2,CONTINUA,249223005,202007,99530.22,-418.29,2604.70,26119.89,1194.66,230,0.00,...,0.743897,0.286208,0.713268,0.695270,0.596098,0.922660,0.203755,0.949999,0.699981,0.139360
3,CONTINUA,249228180,202007,0.00,-22.54,521.80,3758.76,158.00,16,0.00,...,0.235768,0.390517,0.359120,0.273844,0.165481,0.060093,0.203755,0.361661,0.450335,0.388718
4,CONTINUA,249232117,202007,0.00,0.00,1154.58,31466.63,1654.78,29,0.00,...,0.235768,0.665010,0.487548,0.750499,0.708786,0.113705,0.203755,0.491202,0.738943,0.388718


In [ ]:
# Esto corre mal, rehacer
# Agrego Lag y Dlag para las variables

# Columnas que deben excluirse
# exclude_columns = ['numero_de_cliente', 'foto_mes', 'clase_ternaria']

# Iterar sobre las columnas que no están en la lista de exclusión
#for col in data.columns:
#    if col not in exclude_columns:
#        # Crear la columna 'lag' (valor del mes anterior)
#        data[f'{col}_lag'] = data.groupby('numero_de_cliente')[col].shift(1)

        # Crear la columna 'deltaLag' (diferencia entre original y lag)
#        data[f'{col}_deltaLag'] = data[col] - data[f'{col}_lag']

# Verificar el DataFrame resultante
#print(data.head())

  clase_ternaria  numero_de_cliente foto_mes  mpayroll  mcuenta_corriente  \
0       CONTINUA          249221109   202007  64515.00               0.00   
1       CONTINUA          249221468   202007      0.00           -2423.81   
2       CONTINUA          249223005   202007  99530.22            -418.29   
3       CONTINUA          249228180   202007      0.00             -22.54   
4       CONTINUA          249232117   202007      0.00               0.00   

   mrentabilidad  mrentabilidad_annual  mcomisiones  ctrx_quarter  \
0        1550.36              15232.06      1623.85           161   
1         893.83              20716.22       684.18           135   
2        2604.70              26119.89      1194.66           230   
3         521.80               3758.76       158.00            16   
4        1154.58              31466.63      1654.78            29   

   mtransferencias_emitidas  ...  percentile_ctrx_quarter_lag  \
0                  70164.19  ...                         

In [14]:
# Guardar los datos correspondientes a agosto 2021 en un nuevo DataFrame para predicciones, que e slo que va finalmente a kaggle
data_agosto_2021 = data[data['foto_mes'] == "202108"]

In [15]:
# Ahora sí quito julios

# Quito el mes de prueba final, y los julios (el 2020 fue solo para calcular lags y delta lag, y el de 2021 no tiene info de clase ternaria)
data = data[~data['foto_mes'].isin(["202007", "202107", "202108"])]

In [16]:
# Ver los niveles únicos (valores distintos) de la variable 'foto_mes'
print(data['foto_mes'].unique())

# Ver el número de ocurrencias de cada mes en 'foto_mes'
print(data['foto_mes'].value_counts())

# Ver los niveles únicos (valores distintos) de la variable 'foto_mes'
print(data_agosto_2021['foto_mes'].unique())

# Ver el número de ocurrencias de cada mes en 'foto_mes'
print(data_agosto_2021['foto_mes'].value_counts())

['202008' '202009' '202010' '202011' '202012' '202101' '202102' '202103'
 '202104' '202105' '202106']
foto_mes
202106    164876
202105    164623
202104    164090
202103    163685
202102    162646
202101    162026
202012    161526
202011    160742
202010    159731
202009    158371
202008    157058
Name: count, dtype: int64
['202108']
foto_mes
202108    165442
Name: count, dtype: int64


In [17]:
# Parámetros de ganancia
ganancia_acierto = 273000
costo_estimulo = 7000

In [18]:
# Definir los meses de entrenamiento, validación y prueba. Estos son para cuando lo haga entero (saco agosto,que estoyjugado e tiempo)
mes_train = ["202010", "202011",
             "202012", "202101", "202102", "202103", "202104", "202105"]

mes_test = "202106"

Vamos a asignar pesos a las clases. En unos minutos explicaremos las razones detrás de esta decisión. Mientras tanto, pueden aprovechar el código para ajustar el peso de la clase **BAJA+2** según lo deseen.





clase_binaria2: toma el valor 1 si clase_ternaria es diferente de 'CONTINUA'

In [18]:
# Asignación de pesos a las clases
data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

# Crear la clase binaria para el modelo
data['clase_binaria'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)

In [19]:
# Dividir los datos en entrenamiento y prueba
train_data = data[data['foto_mes'].isin(mes_train)].drop(['foto_mes', 'numero_de_cliente'], axis=1)
test_data = data[data['foto_mes'] == mes_test].drop(['foto_mes', 'numero_de_cliente'], axis=1)

NameError: name 'mes_train' is not defined

In [21]:
# Separar características (X), etiquetas (y) y pesos (w) para cada conjunto de datos
X_train = train_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)
y_train = train_data['clase_binaria']
w_train = train_data['clase_peso']

X_test = test_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)
y_test = np.where(test_data['clase_ternaria'] == 'BAJA+2', 1, 0)
w_test = test_data['clase_peso']

In [22]:
# Imputar los valores faltantes en los datos de prueba
imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')
Xif = imp_mean.fit_transform(X_test)

In [ ]:
print(data.shape)
print(train_data.shape)
print(test_data.shape)
print(data_agosto_2021.shape)

(1779374, 65)
(1614498, 63)
(164876, 63)
(165442, 63)


Para evaluar la calidad del modelo, crearemos nuestra propia función de evaluación que calcule la ganancia. La razón de incluir los pesos es precisamente para poder implementar esta función de evaluación de manera adecuada. Al combinar las clases BAJA+1 y BAJA+2 en una sola, necesitamos una forma de diferenciarlas, y es aquí donde entra en juego el weight. Este parámetro nos permitirá distinguir entre ambas clases al momento de evaluarlas dentro del algoritmo.

In [23]:
def calcular_ganancia(y_true, y_pred, weights):
    sorted_indices = np.argsort(y_pred)[::-1]  # Ordenar por las predicciones
    y_true = np.array(y_true)[sorted_indices]
    weights = np.array(weights)[sorted_indices]

    ganancia = np.where(y_true == 1, ganancia_acierto, -costo_estimulo)
    ganancia_acumulada = np.cumsum(ganancia * weights)
    return np.max(ganancia_acumulada)


In [24]:
# Crear datasets para LightGBM
lgb_train = lgb.Dataset(X_train, label=y_train, weight=w_train)

# Generar 500 semillas aleatorias fijas para Optuna
random_seeds = np.random.randint(1, 10**6, size=500)

In [25]:
def objective(trial):
    trial_idx = trial.number
    seed = random_seeds[trial_idx % len(random_seeds)] + trial_idx

    # Configuración de los parámetros del modelo
    params = {
        'objective': 'binary',
        'metric': 'None',
        'boosting_type': 'gbdt',
        'feature_pre_filter': False,
        'num_leaves': trial.suggest_int('num_leaves', 8, 150),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.25),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 5000),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 10.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 10.0),
        'max_depth': trial.suggest_int('max_depth', 3, 50),
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0.0, 1.0),
        'verbose': -1,
        'seed': seed,  # Semilla única
    }

    # Configuración de KFold para la validación cruzada
    kf = KFold(n_splits=3, shuffle=True, random_state=seed)  # Cambiado a 3 folds

    ganancia_promedio = 0
    ganancia_test_acumulada = 0  # Variable para acumular la ganancia en el conjunto de test

    for train_index, val_index in kf.split(X_train):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]
        w_train_fold, w_val_fold = w_train.iloc[train_index], w_train.iloc[val_index]

        # Crear datasets de LightGBM para el fold
        lgb_train_fold = lgb.Dataset(X_train_fold, label=y_train_fold, weight=w_train_fold)
        lgb_val_fold = lgb.Dataset(X_val_fold, label=y_val_fold, weight=w_val_fold, reference=lgb_train_fold)

        # Entrenar el modelo
        model = lgb.train(
            params,
            lgb_train_fold,
            valid_sets=[lgb_train_fold, lgb_val_fold],
            num_boost_round=1000,
            callbacks=[lgb.early_stopping(stopping_rounds=100)],
            feval=lambda y_pred, data: (
                'ganancia',
                calcular_ganancia(data.get_label(), y_pred, data.get_weight()),
                True
            )
        )

        # Predecir en el conjunto de validación
        y_pred_val = model.predict(X_val_fold, num_iteration=model.best_iteration)
        ganancia_fold = calcular_ganancia(y_val_fold, y_pred_val, w_val_fold)
        ganancia_promedio += ganancia_fold

        # Predecir sobre el conjunto de test en cada iteración
        y_pred_test = model.predict(X_test, num_iteration=model.best_iteration)
        ganancia_test_iter = calcular_ganancia(y_test, y_pred_test, w_test)
        ganancia_test_acumulada += ganancia_test_iter

    # Calcular ganancia promedio de los folds
    ganancia_promedio /= 3

    # Calcular la ganancia promedio en el conjunto de prueba
    ganancia_test_promedio = ganancia_test_acumulada / 3

    # Devolver la ganancia en el conjunto de prueba como el objetivo
    return ganancia_test_promedio

In [26]:
# Definir la ubicación del almacenamiento y el nombre del estudio para Optuna
storage_name = "sqlite:///optimization_lgbm.db"
study_name = "exp_lgbm_13"

# Crear y ejecutar el estudio de Optuna
study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True
)

[I 2024-11-24 13:32:19,059] A new study created in RDB with name: exp_lgbm_13


In [27]:
# Configurar Optuna
study.optimize(objective, n_trials = 30)


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[72]	training's ganancia: 3.60967e+08	valid_1's ganancia: 1.71569e+08
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[235]	training's ganancia: 4.65011e+08	valid_1's ganancia: 1.71065e+08
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[141]	training's ganancia: 4.2936e+08	valid_1's ganancia: 1.56659e+08


[I 2024-11-24 13:39:37,161] Trial 0 finished with value: 88011773.15000011 and parameters: {'num_leaves': 16, 'learning_rate': 0.1357159860872724, 'min_data_in_leaf': 1512, 'feature_fraction': 0.3810426403432252, 'bagging_fraction': 0.8984896545750523, 'bagging_freq': 7, 'lambda_l1': 0.7723208613959376, 'lambda_l2': 0.5986243658217172, 'max_depth': 18, 'min_gain_to_split': 0.36272722043910277}. Best is trial 0 with value: 88011773.15000011.


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[11]	training's ganancia: 3.71266e+08	valid_1's ganancia: 1.52256e+08
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[41]	training's ganancia: 4.75645e+08	valid_1's ganancia: 1.57107e+08
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[28]	training's ganancia: 4.29004e+08	valid_1's ganancia: 1.61349e+08


[I 2024-11-24 13:43:37,730] Trial 1 finished with value: 85774173.35666682 and parameters: {'num_leaves': 119, 'learning_rate': 0.19828036245590452, 'min_data_in_leaf': 1474, 'feature_fraction': 0.6678344381746156, 'bagging_fraction': 0.6821147832187847, 'bagging_freq': 5, 'lambda_l1': 3.9547097580802912, 'lambda_l2': 0.31579042672580493, 'max_depth': 13, 'min_gain_to_split': 0.08384859342239614}. Best is trial 0 with value: 88011773.15000011.


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[150]	training's ganancia: 3.92245e+08	valid_1's ganancia: 1.56113e+08
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[293]	training's ganancia: 4.22549e+08	valid_1's ganancia: 1.66473e+08
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[65]	training's ganancia: 3.43523e+08	valid_1's ganancia: 1.7934e+08


...

[I 2024-11-24 16:55:49,507] Trial 27 finished with value: 90384770.39666681 and parameters: {'num_leaves': 117, 'learning_rate': 0.15546539637026457, 'min_data_in_leaf': 4887, 'feature_fraction': 0.9085978681199036, 'bagging_fraction': 0.19543146430362843, 'bagging_freq': 4, 'lambda_l1': 8.411219373884704, 'lambda_l2': 2.9077129947174116, 'max_depth': 27, 'min_gain_to_split': 0.3638217039785973}. Best is trial 12 with value: 93630502.67666668.


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[117]	training's ganancia: 3.66518e+08	valid_1's ganancia: 1.57485e+08
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[170]	training's ganancia: 3.69411e+08	valid_1's ganancia: 1.67607e+08
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[356]	training's ganancia: 3.79e+08	valid_1's ganancia: 1.73914e+08


[I 2024-11-24 17:03:15,975] Trial 28 finished with value: 93373943.27333337 and parameters: {'num_leaves': 134, 'learning_rate': 0.11635336443413329, 'min_data_in_leaf': 4540, 'feature_fraction': 0.9083420838976209, 'bagging_fraction': 0.4400040779261106, 'bagging_freq': 4, 'lambda_l1': 9.535613914488522, 'lambda_l2': 5.907708981757417, 'max_depth': 27, 'min_gain_to_split': 0.5260699433587293}. Best is trial 12 with value: 93630502.67666668.


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[180]	training's ganancia: 3.5179e+08	valid_1's ganancia: 1.69287e+08
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[247]	training's ganancia: 3.664e+08	valid_1's ganancia: 1.60327e+08
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[142]	training's ganancia: 3.47521e+08	valid_1's ganancia: 1.65346e+08


[I 2024-11-24 17:09:53,663] Trial 29 finished with value: 92319346.93333338 and parameters: {'num_leaves': 131, 'learning_rate': 0.04352573063137821, 'min_data_in_leaf': 4359, 'feature_fraction': 0.5463879014509643, 'bagging_fraction': 0.6129738333996151, 'bagging_freq': 5, 'lambda_l1': 9.810673984583286, 'lambda_l2': 1.7168258865714314, 'max_depth': 19, 'min_gain_to_split': 0.7351574904185271}. Best is trial 12 with value: 93630502.67666668.


In [28]:
# Mejor resultado
print(f"Mejor ganancia: {study.best_value}")
print(f"Mejores hiperparámetros: {study.best_params}")

Mejor ganancia: 93630502.67666668
Mejores hiperparámetros: {'num_leaves': 64, 'learning_rate': 0.09003784184754302, 'min_data_in_leaf': 3202, 'feature_fraction': 0.9931996706221722, 'bagging_fraction': 0.12004074552445335, 'bagging_freq': 2, 'lambda_l1': 4.735850108229597, 'lambda_l2': 9.756082154423169, 'max_depth': 35, 'min_gain_to_split': 0.13794398844018246}


In [29]:
plot_param_importances(study)

In [30]:
# Parámetros óptimos obtenidos de la optimización de Optuna
best_iter = study.best_trial.user_attrs["best_iter"]
print(f"Mejor cantidad de árboles para el mejor modelo: {best_iter}")

KeyError: 'best_iter'

Y finalmente tomamos el mejor modelo y lo entrenamos con la totalidad de los
datos

In [ ]:
# Definir los parámetros del modelo con los mejores hiperparámetros encontrados (Se tildó lo de arriba y no quedó guardado. Los pongo a mano abajo)
# params = {
#     'objective': 'binary',
#     'boosting_type': 'gbdt',
#     'first_metric_only': True,
#     'boost_from_average': True,
#     'feature_pre_filter': False,
#     'max_bin': 31,
#     'num_leaves': study.best_trial.params['num_leaves'],
#     'learning_rate': study.best_trial.params['learning_rate'],
#     'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
#     'feature_fraction': study.best_trial.params['feature_fraction'],
#     'bagging_fraction': study.best_trial.params['bagging_fraction'],
#     'max_depth': study.best_trial.params['max_depth'],
#     'lambda_l1': study.best_trial.params['lambda_l1'],
#     'lambda_l2': study.best_trial.params['lambda_l2'],
#     'min_gain_to_split': study.best_trial.params['min_gain_to_split'],
#     'bagging_freq': study.best_trial.params['bagging_freq'],
#     'seed': 11,
#     'verbose': 0
# }

Mejor cantidad de árboles para el mejor modelo: 99
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [20]:
# Definir los parámetros del modelo con los mejores hiperparámetros encontrados
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': 64,
    'learning_rate': 0.09003784184754302,
    'min_data_in_leaf': 3202,
    'feature_fraction': 0.9931996706221722,
    'bagging_fraction': 0.12004074552445335,
    'max_depth': 35,
    'lambda_l1': 4.735850108229597,
    'lambda_l2': 9.756082154423169,
    'min_gain_to_split': 0.13794398844018246,
    'bagging_freq': 2,
    'seed': 11,
    'verbose': 0
}

In [21]:
# Preparación de los datos de entrenamiento (hasta abril 2021) y validación (mayo y junio 2021)
mes_train = ["202008", "202009", "202010", "202011",
             "202012", "202101", "202102", "202103", "202104"]
mes_val = ["202105", "202106"]

# Datos de entrenamiento
train_data = data[data['foto_mes'].isin(mes_train)].drop(['foto_mes', 'numero_de_cliente'], axis=1)
X_train = train_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)
y_train = train_data['clase_binaria']
w_train = train_data['clase_peso']

# Datos de validación (junio 2021)
val_data = data[data['foto_mes'].isin(mes_val)].drop(['foto_mes', 'numero_de_cliente'], axis=1)
X_val = val_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)
y_val = val_data['clase_binaria']
w_val = val_data['clase_peso']


In [22]:
# Crear datasets de LightGBM
lgb_train = lgb.Dataset(X_train, label=y_train, weight=w_train)
lgb_val = lgb.Dataset(X_val, label=y_val, weight=w_val, reference=lgb_train)

In [25]:
# Entrenar el modelo
model = lgb.train(params,
                  lgb_train,
                  num_boost_round=500,
                  valid_sets=[lgb_train, lgb_val],
                  callbacks=[lgb.early_stopping(stopping_rounds=10)])

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits 

In [26]:
importances = model.feature_importance()
feature_names = X_train.columns.tolist()
importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
importance_df = importance_df.sort_values('importance', ascending=False)
importance_df[importance_df['importance'] > 0]


,feature,importance
5,ctrx_quarter,106
13,percentile_mrentabilidad_annual,103
4,mcomisiones,101
7,ctarjeta_visa_transacciones,100
1,mcuenta_corriente,100
14,percentile_mcomisiones,94
9,cproductos,94
11,percentile_mcuenta_corriente,92
2,mrentabilidad,91
3,mrentabilidad_annual,90


In [32]:
# Obtener e imprimir las variables que entraron en el modelo
print("Variables utilizadas en el modelo:")
print(model.feature_name())

Variables utilizadas en el modelo:
['mpayroll', 'mcuenta_corriente', 'mrentabilidad', 'mrentabilidad_annual', 'mcomisiones', 'ctrx_quarter', 'mtransferencias_emitidas', 'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo', 'cproductos', 'percentile_mpayroll', 'percentile_mcuenta_corriente', 'percentile_mrentabilidad', 'percentile_mrentabilidad_annual', 'percentile_mcomisiones', 'percentile_ctrx_quarter', 'percentile_mtransferencias_emitidas', 'percentile_ctarjeta_visa_transacciones', 'percentile_mtarjeta_visa_consumo', 'percentile_cproductos']


In [30]:
print(data_agosto_2021.columns.tolist())

['clase_ternaria', 'numero_de_cliente', 'foto_mes', 'mpayroll', 'mcuenta_corriente', 'mrentabilidad', 'mrentabilidad_annual', 'mcomisiones', 'ctrx_quarter', 'mtransferencias_emitidas', 'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo', 'cproductos', 'percentile_mpayroll', 'percentile_mcuenta_corriente', 'percentile_mrentabilidad', 'percentile_mrentabilidad_annual', 'percentile_mcomisiones', 'percentile_ctrx_quarter', 'percentile_mtransferencias_emitidas', 'percentile_ctarjeta_visa_transacciones', 'percentile_mtarjeta_visa_consumo', 'percentile_cproductos']


In [33]:
# Selección de las variables del modelo
X_agosto_2021 = data_agosto_2021[['mpayroll', 'mcuenta_corriente', 'mrentabilidad', 'mrentabilidad_annual',
                                  'mcomisiones', 'ctrx_quarter', 'mtransferencias_emitidas',
                                  'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo', 'cproductos',
                                  'percentile_mpayroll', 'percentile_mcuenta_corriente',
                                  'percentile_mrentabilidad', 'percentile_mrentabilidad_annual',
                                  'percentile_mcomisiones', 'percentile_ctrx_quarter',
                                  'percentile_mtransferencias_emitidas', 'percentile_ctarjeta_visa_transacciones',
                                  'percentile_mtarjeta_visa_consumo', 'percentile_cproductos']]

# Realizar predicciones para el periodo 202108
y_pred_agosto_2021 = model.predict(X_agosto_2021, num_iteration=model.best_iteration)



In [ ]:
# Calcular la ganancia utilizando las probabilidades predichas
def ganancia_prob(y_pred, y_true, prop=1):
    ganancia = np.where(y_true == 1, ganancia_acierto, 0) - np.where(y_true == 0, costo_estimulo, 0)
    return ganancia[y_pred >= 0.025].sum() / prop

Ganancia LGBM: -105028000.0


In [40]:
# Agregar las probabilidades al DataFrame
data_agosto_2021['pred_prob'] = y_pred_agosto_2021

# Ordenar las predicciones por probabilidad en orden descendente
data_agosto_2021 = data_agosto_2021.sort_values(by='pred_prob', ascending=False)

# Marcar las primeras X predicciones como positivas
X = 6500
data_agosto_2021['Predicted'] = 0  # Inicializar todos como negativos
data_agosto_2021.iloc[:X, data_agosto_2021.columns.get_loc('Predicted')] = 1  # Marcar las primeras X como positivos

# Crear el archivo de salida con las columnas solicitadas
output = data_agosto_2021[['numero_de_cliente', 'Predicted']]

# Guardar el archivo en formato CSV para Kaggle
nombre_archivo = 'predicciones_kaggle_202108.csv'
output.to_csv(nombre_archivo, index=False)
print(f"Archivo {nombre_archivo} guardado.")

# Descargar el archivo para Kaggle
from google.colab import files
files.download(nombre_archivo)


Archivo predicciones_kaggle_202108.csv guardado.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Prediccion de abril

In [37]:
output.shape

(165442, 2)